# Crystal Symmetry

#### Name:

#### Total Points: --/100 pts

In [ ]:
### UNCOMMENT AND RUN THIS CELL IF USING GOOGLE COLAB
# !pip install ipympl -q
# from google.colab import output
# output.enable_custom_widget_manager()


In this notebook, we'll review some of the principles of light propagation in crystals.  In the next notebook, we'll apply these principles to nonlinear optics.

We've already seen that when a cyrstal possesses inversion symmetry, no second-order nonlinear effects can be observed.

Another important symmetry property of crystals is whether it is anisotropic.  An optically isotropic material interacts with light in the same way no matter which polarization the light is.  (Note the unfortunate double-meaning of polarization!)  In contrast, an anisotropic material assumes different properties for light with different polarizations.

Mathematically, this means we need to describe the susceptibility in a way that takes into account.  We write the nonlinear polarization in direction $i$ as:

$$
\begin{align}
P_i = \epsilon_0\left(\sum_j\chi_{ij}^{(1)}E_j + \sum_{jk}\chi_{ijk}^{(2)}E_jE_k + \sum_{jkl}\chi_{ijkl}^{(3)}E_jE_kE_l + \dots \right)
\end{align}
$$

What does this mean?  Well, each subscript corresponds to a cartesion coordinate direction in space (e.g. $x, y, z$).  The linear polarization in the $x$ direction, for example can be expanded as

$$
\begin{align}
P_x = \epsilon_0\left(\chi_{xx}E_x + \chi_{xy}E_y + \chi_{xz}E_z \right) 
\end{align}
$$

indicating that the $P_x$ depends on all three components of the electric field $E_x, E_y$, and $E_z$.  This leads us to write the linear susceptibility in matrix form as

$$
\begin{align}
\chi^{(1)} = 
\begin{bmatrix}
\chi_{xx} & \chi_{xy} & \chi_{xz}\\
\chi_{yx} & \chi_{yy} & \chi_{yz} \\
\chi_{zx} & \chi_{zy} & \chi_{zz}
\end{bmatrix}
\end{align}
$$

making the linear polarization

$$
\begin{align}
P^{(1)} = \epsilon_0
\begin{bmatrix}
\chi_{xx} & \chi_{xy} & \chi_{xz}\\
\chi_{yx} & \chi_{yy} & \chi_{yz} \\
\chi_{zx} & \chi_{zy} & \chi_{zz}
\end{bmatrix}
\begin{bmatrix}
  E_x \\
  E_y \\
  E_z \\
\end{bmatrix}
\end{align}
$$

Likewise, the higher order polarization terms also involve susceptibility tensors.  For example, the second order polarization may be written as a $3\times 3 \times 3$ tensor (think of it as a cube with side length 3) with 27 components.  In general, each of these components is necessary to completely describe the second order nonlinear interaction.  In practice, however, the symmetry properties of the crystals require that most of those values be zero.

Looking at the susceptibility this way allows us to make a mathematical argument for why materials with inversion symmetry cannot have a second order susceptibility.  From above, the second order polarization can be written:

$$
\begin{align}
P_i^{(2)} = \sum_{jk}\chi_{ijk}^{(2)}E_jE_k
\end{align}
$$

If the material has inversion symmetry, then $P_i$ must change sign if all the field components change sign (e.g. the displacement of the electron cloud is in exactly the opposite direction with the same magnitude).  But if both $E_j$ and $E_k$ change signs, then $P_i^{(2)}$ remains unchanged.  Therefore, all elements of $\chi_{ijk}^{(2)}$ are zero.  This same argument applies all even orders of the polarization.

Crystals can be categorized by their symmetry properties, which can be classified by their 32 possible point-symmetry classes.  A point-symmetry class specifies a certain sequance of rotations, reflections, and inversions that leave the crystal strucutre unchanged, and is an entire topic on its own.  As an example, consider a crystal for which the $x$ and $y$ directions are equivalent, but the $z$ direction is diffrent.  This means that if we rotate the crytal by 90 degrees around the z axis, it looks the same.   Such a crystal would have the same optical response for an applied electric field in the $x$ or $y$ direction, so the linear suceptibility $\chi^{(1)}_{zx}$ would be the same as $\chi^{(1)}_{zy}$.  Likewise, the second order susceptibility components $\chi^{(2)}_{zyy}$ would be the same as $\chi^{(2)}_{zxx}$, and $\chi^{(2)}_{zxy}$ would be equal to $\chi^{(2)}_{zyx}$.  By this process, we can understand which elements of a nonlinear susceptibility will be equal and/or nonzero by looking at its symmetry classification.  Crystallographers have identified 7 classes of crystals.

In python, we can easily look up the symmetry properties of any crystal and explore its properties usin the crystals package.  For example, say we wanted to explore diamond as an optical material.  We can load up it's crystal properties as follows:

In [ ]:
from crystals import Crystal
diamond = Crystal.from_database('diamond')
diamond

This creates a crystal object, which contains the number of atoms in a unit cell and their relative position.  This object also contains the space-group properties:

In [ ]:
diamond.lattice_system

This tells us that diamond belongs to the cubic system, which is also number 7 in the usual classification list.  Further more specific information is also available:

In [ ]:
diamond.symmetry()

This tells us that diamond is a face centered cubic crystal with m-3m point group. This means that it has a mirror plane (m) parllel to one axis, and and a 3-fold axis of symmetry and a mirror plane parallel to another axis.  Typically, most practitioners of nonlinear optics do no memorize all these point-groups, but use the classifications provided by crystallogrophers to select crystals for nonlinear optics experiments.

We can also visualize the crystal structure using our diamond crystal object:

In [ ]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.pyplot as plt 
from scipy.spatial.distance import pdist, squareform
%matplotlib widget

L = np.array(diamond.supercell(2,2,2))
N = len(L)
X = L[:,1:4]
k =3

# matrix of pairwise Euclidean distances
distmat = squareform(pdist(X, 'euclidean'))

# select the kNN for each datapoint
neighbors = np.sort(np.argsort(distmat, axis=1)[:, 0:k])

# get edge coordinates
coordinates = np.zeros((N, k, 2, 3))
for i in np.arange(N):
    for j in np.arange(k):
        coordinates[i, j, :, 0] = np.array([X[i,:][0], X[neighbors[i, j], :][0]])
        coordinates[i, j, :, 1] = np.array([X[i,:][1], X[neighbors[i, j], :][1]])
        coordinates[i, j, :, 2] = np.array([X[i,:][2], X[neighbors[i, j], :][2]])


fig, ax = plt.subplots(1,1,figsize = (8, 8))
#fig = plt.figure()
ax = fig.add_subplot(111, projection='3d') 
ax.scatter(X[:,0], X[:,1], X[:,2], s = 100)
for i in np.arange(N):
    for j in np.arange(k):
        ax.plot(coordinates[i,j,:,0], coordinates[i,j,:,1], coordinates[i,j,:,2], color = 'black') 
plt.show()

Ok, hopefully you are convinced that crystal structure is important to understanding the linear and nonlinear optical response of a material.  Let's now turn our attention to how light propogates in anisotropic media.  For the purposes of optics, this means that the atomic polarization will be differnt for differnt applied field components.

Recall that when we derived the nonlinear wave equation from Maxwell's equation that the polarization $P$ came into play when we introduced the contituitive relation $\tilde{\mathbf{D}} = \epsilon_0 \tilde{\mathbf{E}} + \tilde{\mathbf{P}}$.  Let's just consider linear materials for the moment.  In this case we can write

$$
\begin{align}
\tilde{\mathbf{D}} = \epsilon_0\mathbf{\epsilon}\tilde{\mathbf{E}}
\end{align}
$$

where we now allow the dielectric constant $\mathbf{\epsilon}$ to depend on the polarizatio of the field.  W can write this in matrix form:

$$
\begin{align}
\begin{bmatrix}
  D_x \\
  D_y \\
  D_z \\
\end{bmatrix}
= \epsilon_0
\begin{bmatrix}
\epsilon_{xx} & \epsilon_{xy} & \epsilon_{xz}\\
\epsilon_{yx} & \epsilon_{yy} & \epsilon_{yz} \\
\epsilon_{zx} & \epsilon_{zy} & \epsilon_{zz}
\end{bmatrix}
\begin{bmatrix}
  E_x \\
  E_y \\
  E_z \\
\end{bmatrix}
\end{align}
$$

It appears again that we have 9 individual dielectric constants!  But since the material is linear, we can simply use linear algebra to diagonalize this matrix and define a new coordinate system such that 

$$
\begin{align}
\begin{bmatrix}
  D_x \\
  D_y \\
  D_z \\
\end{bmatrix}
= \epsilon_0
\begin{bmatrix}
\epsilon_{xx} & 0 & 0\\
0 & \epsilon_{yy} & 0 \\
0& 0 & \epsilon_{zz}
\end{bmatrix}
\begin{bmatrix}
  E_x \\
  E_y \\
  E_z \\
\end{bmatrix}
\end{align}
$$

These new coordinates define the principal axis of the material.  If all three principal dielectric constants ($\epsilon_{xx}, \epsilon_{yy}, \epsilon_{zz}$) are the same, we have an isotropic material.  If two of them are the same but one is different, the material is uniaxial, and if all three are different it is biaxial.

Most commonly, researchers use uniaxial crystals.  In this case, the material will have two refractive indices and ordinary refractive index $n_o = n_x = n_y $ and an extraordinatry refractive index $n_e = n_z$

## Exercises

1. (100 pts) Look up the 7 crystal classes (Triclinic, Monoclini, Orthorhombic, Tetragonal, Trigonal, Hexagonal, Cubic, Isotropic) and write their linear susceptibility tensors in matrix form indicating which elements are nonzero.  Indicate whether each class is an isotropic, uniaxial, or biaxial crystal, and why. Give an example of that crystal and an interesting fact about it.


### Example: Monoclinic

$$
\begin{bmatrix}
\chi_{xx} & 0 & \chi_{xz}\\
0 & \chi_{yy} & 0 \\
\chi_{zx} & 0 & \chi_{zz}
\end{bmatrix}
$$

Monoclinic crystals are biaxial since there is not one particular direction that displays rotational symmetry.  In other words, diagonlizing the above matrix would always result in 3 different values along the diagonal.  An example is muscovite (mica), which shows strong birefrigence, and as a result "glitters" and is commonly used in cosmetics such as lipstick and nail polish to add luster.   

2. For each of the example materials you choose, find the unit cell and symmetry properties using the Crystals package in Python.

3. Choose one each of an isotropic, uniaxial, and biaxial crystal that are optically transparent.  Look up the principal indices of refraction and displace them in matrix form.

4. Suppose you wanted to make a half-waveplate for light at 633 nm out of the uniaxial crystal you chose. What thickness would you choose?  Why?

